# Deep Learning in Medicine
### BMSC-GA 4493, BMIN-GA 3007 
### Lab 6: Regularization




### Goal of this lab: 
    - Understand Regularization Methods 
    - Apply them in the example MNIST classification problem

### Questions: 
    - What is regularization?
    - What problem is regularization trying to solve?
    - What are some of the regularization methods?

In [ ]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Parameters
batch_size = 20
test_batch_size = 20
training_epoches = 20

In [ ]:
torch.manual_seed(717)
use_cuda = torch.cuda.is_available()
kwargs = {}

if use_cuda:
    torch.cuda.manual_seed(717)
    kwargs = {'num_workers': 1, 'pin_memory': True} 

   

In [ ]:
# Data Loader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [ ]:
# Simple Neural Network Design
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)


In [ ]:
# Training Steps
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 5000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        train_loss += loss.data[0]
    train_loss /=len(train_loader)
    return train_loss

In [ ]:
# Validation/Test Steps
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))
    
    return test_loss


In [ ]:
model = Net()
if use_cuda:
    model.cuda()
#Optimizor 
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.5)

train_losses =[]
test_losses =[]
for epoch in range(1, training_epoches + 1):
    train_losses.append(train(epoch))
    test_losses.append(test(epoch, test_loader))

In [ ]:
epoch_number = list(range(1, training_epoches + 1))
plt.plot(epoch_number,train_losses, 'r' )
plt.plot(epoch_number,test_losses, 'b' )
plt.show()

### Question: What does this graph show? And how to solve it?

## Method 1: Add Dropout Layers
Reference: Dropout: A Simple Way to Prevent Neural Networks from Overfitting, https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf

In [ ]:
# Please add your code here
# Hint: Add dropout layers in network design



In [ ]:
# See your results here
model = Net()
if use_cuda:
    model.cuda()
#Optimizor 
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.5)

train_losses =[]
test_losses =[]
for epoch in range(1, training_epoches + 1):
    train_losses.append(train(epoch))
    test_losses.append(test(epoch, test_loader))

epoch_number = list(range(1, training_epoches + 1))
plt.plot(epoch_number,train_losses, 'r' )
plt.plot(epoch_number,test_losses, 'b' )
plt.show()

## Method 2: Add Penalty in Loss Function


## Method 3: Early Stop
Reference: Early stop: https://en.wikipedia.org/wiki/Early_stopping

In [ ]:
# Please add your code here
# Hint: for example, stop training when the test loss is not improving



Reference of this exercise: https://github.com/pytorch/examples/blob/master/mnist/main.py